##Installing DeepChem

In [1]:
!curl -Lo conda_installer.py https://raw.githubusercontent.com/deepchem/deepchem/master/scripts/colab_install.py
import conda_installer
conda_installer.install()
!/root/miniconda/bin/conda info -e

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3490  100  3490    0     0  12973      0 --:--:-- --:--:-- --:--:-- 12973


add /root/miniconda/lib/python3.6/site-packages to PYTHONPATH
python version: 3.6.9
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit, openmm, pdbfixer
added conda-forge to channels
added omnia to channels
done
conda packages installation finished!


# conda environments:
#
base                  *  /root/miniconda



In [2]:
!pip install --pre deepchem

     |████████████████████████████████| 542kB 9.0MB/s 


We can now import the `deepchem` package to play with.

In [3]:
import deepchem as dc
dc.__version__

'2.5.0.dev'

## Baseline - Fingerprints + NN

Implementing and recording the baseline for Tox21 dataset.

In [4]:
tasks, datasets, transformers = dc.molnet.load_lipo(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets
print(train_dataset)

<DiskDataset X.shape: (3360, 1024), y.shape: (3360, 1), w.shape: (3360, 1), task_names: ['exp']>


ECFP featurizer is used. Extended Connectivity Fingerprints  is a **fingerprinting** method. They are also sometimes called "circular fingerprints". The ECFP algorithm begins by classifying atoms based only on their direct properties and bonds. Each unique pattern is a feature. For example, "carbon atom bonded to two hydrogens and two heavy atoms" would be a feature, and a particular element of the fingerprint is set to 1 for any molecule that contains that feature. It then iteratively identifies new features by looking at larger circular neighborhoods. One specific feature bonded to two other specific features becomes a higher level feature, and the corresponding element is set for any molecule that contains it. This continues for a fixed number of iterations, most often two.

In [5]:
tasks

['exp']

Above is the task in Lipo dataset -  we have 1 tasks, corresponding to different lipophilicity targets, such as cell receptors and stress response pathways.

In [6]:
datasets

(<DiskDataset X.shape: (3360, 1024), y.shape: (3360, 1), w.shape: (3360, 1), task_names: ['exp']>,
 <DiskDataset X.shape: (420, 1024), y.shape: (420, 1), w.shape: (420, 1), ids: ['Nc1nccc(\\C=C\\c2ccc(Cl)cc2)n1'
  'CC(=O)Nc1ccc2ccn(c3cc(NCCCN4CCCC4)n5ncc(C#N)c5n3)c2c1'
  'OC(CN1CCOCC1)c2ccccc2' ...
  'COc1cc(CCc2cc(Nc3ccnc(NCc4ccccn4)n3)n[nH]2)cc(OC)c1'
  'CCN(CC)C(=O)c1ccc(cc1)C(=C2CCN(Cc3ccc(F)cc3)CC2)c4cccnc4'
  'CC(C)(C(=O)Nc1oc(nn1)C(=O)Nc2ccc(cc2)N3CCOCC3)c4ccc(Cl)cc4'], task_names: ['exp']>,
 <DiskDataset X.shape: (420, 1024), y.shape: (420, 1), w.shape: (420, 1), ids: ['O[C@@H](CNCCCOCCNCCc1cccc(Cl)c1)c2ccc(O)c3NC(=O)Sc23'
  'NC1=NN(CC1)c2cccc(c2)C(F)(F)F'
  'COc1cc(ccc1Cn2ccc3ccc(cc23)C(=O)NCC4CCCC4)C(=O)NS(=O)(=O)c5ccccc5' ...
  'CCCSc1ncccc1C(=O)N2CCCC2c3ccncc3' 'Oc1ncnc2scc(c3ccsc3)c12'
  'OC1(CN2CCC1CC2)C#Cc3ccc(cc3)c4ccccc4'], task_names: ['exp']>)

There are three dataset objects - train split, val split and test split. Each split consists of X and y - X is the features and y is the output label. Froe example the train split has X.shape (3360, 1024)
and y.shape (3360, 1). This implies that there are 3360 samples in the train split - and each sample is represented by an ECFP vector of size 1024. 

##Training a Model on Fingerprints

In [24]:
model = dc.models.MultitaskClassifier(n_tasks=1, n_features=1024, layer_sizes=[1000])

`MultitaskClassifier` is a simple stack of fully connected layers. A single hidden layer of width 1000 is used. Each input will have 1024 features, and it should produce predictions for 1 task.

Note that the above network is performing learning - a single network is used for the task. This is because inter task correlations exist in the data, and to take if advantage of this single neural network is used for multiple tasks.

In [25]:
import numpy as np

model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print('training set score:', model.evaluate(train_dataset, [metric], transformers))
print('test set score:', model.evaluate(test_dataset, [metric], transformers))

ValueError: ignored

The training set score is much higher than test set score. This indicates overfitting - and is why metrics on the validation set need to be measured in otder to tune parameters and detect overfitting.